# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-674433840b-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# GEO Workflow Series ("European Cathedrals") 

Consider the following exploratory information need:

> Explore how many cathedrals exists in the U.K., Italy, and France and the information about them

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate |
| `wdt:P17`     | country   | predicate |
| `wd:Q2977`    | Cathedral     | node      |
| `wd:Q29265`   | Canterbury Cathedral   | node |
| `wd:Q38`      | Italy         | node |
| `wd:Q142`     | France        | node |
| `wd:Q145`     | U.K.          | node |
| `wd:Q46261`   | Romanesque architecture | node |


Also consider

```
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q2977  . 
```

is the BGP to retrieve all **French cathedrals**

## Workload Goals

1. Identify the BGP to obtain relevant attributes of a cathedral, e.g., the inception or the height

2. Identify the BGP that connect each cathedreal to other relevant entities, e.g., architectural style and city

3. Which country hosts the largest number of cathedrals?

4. Analyze cathedrals across architectural styles in each city
 
   4.1 Which styles exists in each country, and how many cathedrals for each one?
   
   4.2 where is the largest cathedral, or the oldest?
   
   4.3 If you had to pick a city or a country to visit some cathedrals? Which one would you choose?  Based on what criteria?


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q2977  . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '87')]


1

## Part 1
### Identify the BGP to obtain relevant attributes of a cathedral, e.g., the inception or the height

### Find information about wd:Q29265 Canterbury Cathedral

In [4]:
queryString = """
SELECT DISTINCT ?pName ?oName ?p ?o
WHERE { 
    wd:Q29265 ?p ?o .
    OPTIONAL { ?p sc:name ?pName .}
    OPTIONAL { ?o sc:name ?oName .}   
} ORDER BY ?pName ?p

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://schema.org/name'), ('o', 'Canterbury Cathedral')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1004'), ('o', 'http://musicbrainz.org/place/b88cb98c-a4f8-45a1-973c-567e995c311f')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1015'), ('o', 'https://livedata.bibsys.no/authority/90549142')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1566'), ('o', 'http://sws.geonames.org/6619924/')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1667'), ('o', 'http://vocab.getty.edu/tgn/1100072')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('o', 'http://viaf.org/viaf/122308660')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163'), ('o', 'http://id.worldcat.org/fast/550506')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227'), ('o', 'https://d-nb.info/gnd/4069841-5')]
[('p', 'http://www.wikidata.org/p

100

### Identify the BGP to obtain relevant attributes of a cathedrals
[('pName', 'height'), ('p', 'http://www.wikidata.org/prop/direct/P2048'), ('o', '72')]

[('pName', 'inception'), ('p', 'http://www.wikidata.org/prop/direct/P571'), ('o', '0597-01-01T00:00:00Z')]

[('pName', 'country'), ('oName', 'United Kingdom'), ('p', 'http://www.wikidata.org/prop/direct/P17'),

In [5]:
queryString = """
SELECT DISTINCT ?cathName ?countryName ?height ?costructionYear
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P2048 ?height ;
          wdt:P571 ?inception ;
          wdt:P17 ?country ;
          sc:name ?cathName .
    BIND(year(xsd:dateTime(?inception)) AS ?costructionYear)
    ?country sc:name ?countryName .
}
ORDER BY DESC(?height)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Turku Cathedral'), ('countryName', 'Finland'), ('height', '10190'), ('costructionYear', '1300')]
[('cathName', 'Cathedral of Saint Paul, National Shrine of the Apostle Paul'), ('countryName', 'United States of America'), ('height', '306.5'), ('costructionYear', '1906')]
[('cathName', 'New Cathedral, Linz'), ('countryName', 'Austria'), ('height', '134.7'), ('costructionYear', '1862')]
[('cathName', 'Cathedral of Maringá'), ('countryName', 'Brazil'), ('height', '124'), ('costructionYear', '1972')]
[('cathName', 'Freiburg Minster'), ('countryName', 'Germany'), ('height', '116'), ('costructionYear', '1200')]
[('cathName', 'Florence Cathedral'), ('countryName', 'Italy'), ('height', '114.5'), ('costructionYear', '1296')]
[('cathName', 'Cathedral Basilica of St. James the Apostle'), ('countryName', 'Poland'), ('height', '110.18'), ('costructionYear', '1101')]
[('cathName', 'Archcathedral Basilica of St. Stanislaus Kostka, Łódź'), ('countryName', 'Poland'), ('height', '1

20

### Result part 1
I decided to order the cathedrals based on their height... it is clear that the first one has a wrong data about its height.

### Find more informations about wrong data: 
### See cathedrals with wrong height

In [6]:
queryString = """
SELECT DISTINCT ?cathName ?countryName ?height ?costructionYear
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P2048 ?height ;
          wdt:P571 ?inception ;
          wdt:P17 ?country ;
          sc:name ?cathName .
    BIND(year(xsd:dateTime(?inception)) AS ?costructionYear)
    ?country sc:name ?countryName .

    FILTER (?height > 200)
}
ORDER BY DESC(?height)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Turku Cathedral'), ('countryName', 'Finland'), ('height', '10190'), ('costructionYear', '1300')]
[('cathName', 'Cathedral of Saint Paul, National Shrine of the Apostle Paul'), ('countryName', 'United States of America'), ('height', '306.5'), ('costructionYear', '1906')]


2

### Verify if ?height data is stored as literal

In [7]:
queryString = """
ASK WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P2048 ?height ;
          wdt:P571 ?inception ;
          wdt:P17 ?country ;
          sc:name ?cathName .
    BIND(year(xsd:dateTime(?inception)) AS ?costructionYear)
    ?country sc:name ?countryName .

    FILTER isLiteral(?height)
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

### Conclusion of wrong height data
Height data are stored as literal so we are not able to detect the unit of measurement... the Turku Cathedral has surely a wrong data but Cathedral of Saint Paul, National Shrine of the Apostle Paul could have a correct height data but expressed in feet and not in meters.

## Part 2
### Identify the BGP that connect each cathedreal to other relevant entities, e.g., architectural style and city

Examples
[('pName', 'architectural style'), ('oName', 'Gothic art'), ('p', 'http://www.wikidata.org/prop/direct/P149'), ('o', 'http://www.wikidata.org/entity/Q46825')]

[('pName', 'located in the administrative territorial entity'), ('oName', 'Canterbury'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('o', 'http://www.wikidata.org/entity/Q1094104')]

In [8]:
queryString = """
SELECT DISTINCT ?cathName ?locationName ?countryName (GROUP_CONCAT(DISTINCT ?styleName; SEPARATOR="; ") AS ?styles)
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P149 ?style ;
          wdt:P131 ?location ;
          wdt:P17 ?country ;
          sc:name ?cathName .
          
    OPTIONAL{?style sc:name ?styleName .}
    OPTIONAL{?location sc:name ?locationName .}
    ?country sc:name ?countryName .
}
GROUP BY ?cathName ?locationName ?countryName 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Church of St. Vincent'), ('locationName', 'Wrocław'), ('countryName', 'Poland'), ('styles', 'Brick Gothic')]
[('cathName', 'Brechin Cathedral'), ('locationName', 'Angus'), ('countryName', 'United Kingdom'), ('styles', 'Gothic art')]
[('cathName', 'Cathedral'), ('locationName', 'Bitetto'), ('countryName', 'Italy'), ('styles', 'Romanesque architecture')]
[('cathName', 'Cathedral of Our Lady of Nahuel Huapi'), ('locationName', 'Bariloche'), ('countryName', 'Argentina'), ('styles', 'Gothic Revival')]
[('cathName', 'Basilica of St. Joseph Proto-Cathedral'), ('locationName', 'Bardstown'), ('countryName', 'United States of America'), ('styles', 'Greek Revival architecture')]
[('cathName', 'Cathedral of the Holy Trinity, Mostar'), ('locationName', 'Mostar'), ('countryName', 'Bosnia and Herzegovina'), ('styles', 'eclectic architecture')]
[('cathName', 'Cathedral of Our Lady of the Assumption, Oaxaca'), ('locationName', 'Oaxaca de Juárez'), ('countryName', 'Mexico'), ('sty

20

### Results part 2
We can notice that there are cathedrals with more than one architectural style.

### Some additional interesting queries:
### Search if exists a cathedral without information about its location

In [9]:
queryString = """
SELECT DISTINCT ?cathName ?countryName
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P17 ?country ;
          sc:name ?cathName .

    ?country sc:name ?countryName .
    FILTER (NOT EXISTS {?cath  wdt:P131 ?location .} )
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Medhane Alem Cathedral (Harar)'), ('countryName', 'Ethiopia')]
[('cathName', 'Wesley Methodist Cathedral'), ('countryName', 'Ghana')]
[('cathName', 'The Corona, Canterbury Cathedral'), ('countryName', 'United Kingdom')]
[('cathName', 'St. Colombo Cathedral'), ('countryName', "People's Republic of China")]
[('cathName', "Saint Joseph's Cathedral"), ('countryName', "People's Republic of China")]
[('cathName', 'Cathedral of Shuozhou'), ('countryName', "People's Republic of China")]
[('cathName', 'Cathedral of the Holy Cross'), ('countryName', "People's Republic of China")]
[('cathName', 'Franciscan Church of the Five Injuries of Dongshan'), ('countryName', "People's Republic of China")]
[('cathName', 'Catedral Diocesana'), ('countryName', 'Brazil')]
[('cathName', 'Holy Trinity Cathedral'), ('countryName', 'Russia')]
[('cathName', 'The Cathedral of the Holy and Righteous Warrior Feodor Ushakov'), ('countryName', 'Russia')]
[('cathName', 'Trinity Cathedral, Trubchevsk

20

### Search if exists a cathedral without information about its location and its country

In [10]:
queryString = """
SELECT DISTINCT ?cathName
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          sc:name ?cathName .

    FILTER (NOT EXISTS {?cath  wdt:P131 ?location .} && NOT EXISTS {?cath  wdt:P17 ?country .})
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
Empty


0

### Search if exists a cathedral without information about its country

In [11]:
queryString = """
SELECT DISTINCT ?cathName
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          sc:name ?cathName .

    FILTER ( NOT EXISTS {?cath  wdt:P17 ?country .})
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
Empty


0

### Conclusion
Every cathedral has information about its country, but some of them have not information about their exact location

## Part 3
### Which country hosts the largest number of cathedrals?

In [12]:
queryString = """
SELECT DISTINCT ?countryName (COUNT(DISTINCT ?cath) AS ?cathNumber)
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P17 ?country .
          
    ?country sc:name ?countryName .
}
GROUP BY ?countryName
ORDER BY DESC(?cathNumber)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Italy'), ('cathNumber', '293')]
[('countryName', 'United States of America'), ('cathNumber', '128')]
[('countryName', 'Russia'), ('cathNumber', '94')]
[('countryName', 'France'), ('cathNumber', '87')]
[('countryName', 'Canada'), ('cathNumber', '67')]
[('countryName', 'Mexico'), ('cathNumber', '64')]
[('countryName', 'Brazil'), ('cathNumber', '57')]
[('countryName', "People's Republic of China"), ('cathNumber', '52')]
[('countryName', 'Philippines'), ('cathNumber', '48')]
[('countryName', 'United Kingdom'), ('cathNumber', '47')]
[('countryName', 'Ukraine'), ('cathNumber', '44')]
[('countryName', 'India'), ('cathNumber', '41')]
[('countryName', 'Poland'), ('cathNumber', '35')]
[('countryName', 'Germany'), ('cathNumber', '34')]
[('countryName', 'Argentina'), ('cathNumber', '32')]
[('countryName', 'Colombia'), ('cathNumber', '32')]
[('countryName', 'Romania'), ('cathNumber', '29')]
[('countryName', 'Greece'), ('cathNumber', '27')]
[('countryName', 'Spain'), ('cath

20

### Result part 3
It is surprising that Italy is the country with highest number of cathedrals (in our database)

## Part 4
### Analyze cathedrals across architectural styles in each city

### Part 4.1
Which styles exists in each country, and how many cathedrals for each one?

In [13]:
queryString = """
SELECT ?countryName ?archName (COUNT(DISTINCT ?cath) AS ?cathNumber)
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
        wdt:P149 ?style ;
        wdt:P17 ?country .
          
    ?country sc:name ?countryName .
    
    ?style sc:name ?archName .
    
}
GROUP BY ?countryName ?archName
ORDER BY ?countryName ?archName
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Albania'), ('archName', 'Byzantine Revival architecture'), ('cathNumber', '1')]
[('countryName', 'Albania'), ('archName', 'Byzantine art'), ('cathNumber', '1')]
[('countryName', 'Albania'), ('archName', 'Romanesque Revival architecture'), ('cathNumber', '1')]
[('countryName', 'Algeria'), ('archName', 'Islamic architecture'), ('cathNumber', '1')]
[('countryName', 'Angola'), ('archName', 'Gothic Revival'), ('cathNumber', '1')]
[('countryName', 'Angola'), ('archName', 'Neoclassical architecture'), ('cathNumber', '1')]
[('countryName', 'Angola'), ('archName', 'modern architecture'), ('cathNumber', '1')]
[('countryName', 'Argentina'), ('archName', 'Baroque Revival architecture'), ('cathNumber', '1')]
[('countryName', 'Argentina'), ('archName', 'Gothic Revival'), ('cathNumber', '7')]
[('countryName', 'Argentina'), ('archName', 'Gothic architecture'), ('cathNumber', '1')]
[('countryName', 'Argentina'), ('archName', 'Italianate architecture'), ('cathNumber', '1')]
[('

20

### Group the information for each contry 

In [14]:
queryString = """
SELECT ?countryName (GROUP_CONCAT(DISTINCT ?archInfo; SEPARATOR=", ") AS ?countryData)
WHERE{
    {
        SELECT ?countryName ?archName (COUNT(DISTINCT ?cath) AS ?cathNumber)
        WHERE { 
        ?cath wdt:P31 wd:Q2977 ;
            wdt:P149 ?style ;
            wdt:P17 ?country .
            
        ?country sc:name ?countryName .
        
        ?style sc:name ?archName .
        
        }
        GROUP BY ?countryName ?archName
    }
    BIND(CONCAT(?archName,": ",?cathNumber) AS ?archInfo)
}
GROUP BY ?countryName
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Sweden'), ('countryData', 'Romanesque Revival architecture: 1')]
[('countryName', 'Iran'), ('countryData', 'Isfahani style: 1, Pahlavi architecture: 2, Safavid architecture: 1')]
[('countryName', 'Cyprus'), ('countryData', 'Gothic architecture: 2')]
[('countryName', 'Czech Republic'), ('countryData', 'Baroque: 1, Gothic Revival: 1, Gothic architecture: 1, Gothic art: 1, baroque architecture: 4, baroque gothic: 1')]
[('countryName', 'Philippines'), ('countryData', 'Baroque: 1, Earthquake Baroque: 1, Gothic Revival: 2, Italian Renaissance: 1, Neoclassical architecture: 3, Romanesque Revival architecture: 1, Romanesque architecture: 2, baroque architecture: 7, modern architecture: 2')]
[('countryName', 'Vanuatu'), ('countryData', 'modern architecture: 1')]
[('countryName', 'India'), ('countryData', 'Baroque: 1, Colonial architecture: 1, Gothic Revival: 7, Gothic architecture: 2, Gothic art: 1, baroque architecture: 1')]
[('countryName', 'Brazil'), ('countryData',

30

### Order by number of architectural styles present in a country

In [15]:
queryString = """
SELECT ?countryName (GROUP_CONCAT(DISTINCT ?archInfo; SEPARATOR=", ") AS ?countryData)
WHERE{
    {
        SELECT ?countryName ?archName (COUNT(DISTINCT ?cath) AS ?cathNumber)
        WHERE { 
        ?cath wdt:P31 wd:Q2977 ;
            wdt:P149 ?style ;
            wdt:P17 ?country .
            
        ?country sc:name ?countryName .
        
        ?style sc:name ?archName .
        
        }
        GROUP BY ?countryName ?archName
    }
    BIND(CONCAT(?archName,": ",?cathNumber) AS ?archInfo)
}
GROUP BY ?countryName
ORDER BY DESC(COUNT(DISTINCT ?archInfo))
LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Italy'), ('countryData', 'Baroque: 4, Byzantine architecture: 3, Chiaramonte style: 1, First Romanesque: 1, Gothic Revival: 5, Gothic architecture: 20, Gothic art: 2, International Gothic: 2, Italian Baroque architecture: 3, Italian Gothic architecture: 2, Italian Neoclassical architecture: 1, Mannerism: 3, Mannerist architecture: 2, Napolitan baroque architecture: 1, Neoclassical architecture: 15, Norman architecture: 5, Norman-Arab-Byzantine culture: 3, Pre-Romanesque architecture: 1, Renaissance Revival architecture: 2, Renaissance architecture: 17, Renaissance: 5, Romanesque Revival architecture: 5, Romanesque architecture: 82, Romanesque art: 3, Sicilian Baroque: 4, barocco leccese: 1, baroque architecture: 52, classicism: 1, medieval architecture: 1, modern architecture: 1, neoclassicism: 9, paleochristian architecture: 1, rationalism: 2')]
[('countryName', 'United States of America'), ('countryData', 'Armenian architecture: 1, Art Deco: 1, Baroque Reviv

30

### Results part 4.1
Some countries have only one architectural style... but a lot of countries have more than one architectural style.

As expected (considering the results of the previous queries) Italy is the country with the higher number of architectural styles and number of cathedrals

### Part 4.2
Where is the largest cathedral, or the oldest?

#### Oldest cathedrals

In [16]:
queryString = """
SELECT DISTINCT ?cathName ?countryName ?costructionYear
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P571 ?inception ;
          wdt:P17 ?country ;
          sc:name ?cathName .
    BIND(year(xsd:dateTime(?inception)) AS ?costructionYear)
    ?country sc:name ?countryName .
}
ORDER BY ASC(?costructionYear)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Etchmiadzin Cathedral'), ('countryName', 'Armenia'), ('costructionYear', '303')]
[('cathName', 'Cathedral of Saint Mary of the Assumption (Chur)'), ('countryName', 'Switzerland'), ('costructionYear', '401')]
[('cathName', 'Pula Cathedral'), ('countryName', 'Croatia'), ('costructionYear', '401')]
[('cathName', 'Bolnisi Sioni'), ('countryName', 'Georgia'), ('costructionYear', '401')]
[('cathName', 'Krk Cathedral'), ('countryName', 'Croatia'), ('costructionYear', '501')]
[('cathName', 'Duomo of Monza'), ('countryName', 'Italy'), ('costructionYear', '501')]
[('cathName', 'Luni Cathedral'), ('countryName', 'Italy'), ('costructionYear', '501')]
[('cathName', 'Ninotsminda Cathedral'), ('countryName', 'Georgia'), ('costructionYear', '501')]
[('cathName', 'Euphrasian Basilica'), ('countryName', 'Croatia'), ('costructionYear', '553')]
[('cathName', 'Cathedral of Talin'), ('countryName', 'Armenia'), ('costructionYear', '601')]


10

### Display the oldest cathedral present in our database

In [17]:
queryString = """
SELECT DISTINCT ?cathName ?countryName ?costructionYear
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P571 ?inception ;
          wdt:P17 ?country ;
          sc:name ?cathName .
    BIND(year(xsd:dateTime(?inception)) AS ?costructionYear)
    ?country sc:name ?countryName .
}
ORDER BY ASC(?costructionYear)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Etchmiadzin Cathedral'), ('countryName', 'Armenia'), ('costructionYear', '303')]


1

### Display the largest cathedrals present in our database (consider the area data as dimension)

In [18]:
queryString = """
SELECT DISTINCT ?cathName ?countryName ?area
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P2046 ?area ;
          wdt:P2048 ?height ;
          wdt:P17 ?country ;
          sc:name ?cathName .
    ?country sc:name ?countryName .

}
ORDER BY DESC(?length)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Katholische Hofkirche'), ('countryName', 'Germany'), ('area', '4800')]
[('cathName', 'Florence Cathedral'), ('countryName', 'Italy'), ('area', '8600')]
[('cathName', 'Tournai Cathedral'), ('countryName', 'Belgium'), ('area', '20.2219')]
[('cathName', 'Tournai Cathedral'), ('countryName', 'Belgium'), ('area', '0.4963')]


4

### Display the largest cathedrals present in our database (consider the lenght data as dimension)

In [19]:
queryString = """
SELECT DISTINCT ?cathName ?countryName ?length
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P2043 ?length ;
          wdt:P17 ?country ;
          sc:name ?cathName .
    ?country sc:name ?countryName .

}
ORDER BY DESC(?length)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('cathName', 'Mary, Queen of the World Cathedral'), ('countryName', 'Canada'), ('length', '10150')]
[('cathName', "Old St Paul's Cathedral"), ('countryName', 'United Kingdom'), ('length', '586')]
[('cathName', 'Cathedral of Saint Paul, National Shrine of the Apostle Paul'), ('countryName', 'United States of America'), ('length', '307')]
[('cathName', "Old St Paul's Cathedral"), ('countryName', 'United Kingdom'), ('length', '179')]
[('cathName', 'York Minster'), ('countryName', 'United Kingdom'), ('length', '160')]
[('cathName', 'Florence Cathedral'), ('countryName', 'Italy'), ('length', '153')]
[('cathName', 'Church of the Holy Sepulchre'), ('countryName', 'State of Palestine'), ('length', '120')]
[('cathName', 'Church of the Holy Sepulchre'), ('countryName', 'Israel'), ('length', '120')]
[('cathName', 'Esztergom Basilica'), ('countryName', 'Hungary'), ('length', '120')]
[('cathName', 'Cathedral of St. Michael and St. Gudula'), ('countryName', 'Belgium'), ('length', '114')]


10

### Display the largest cathedral present in our database (consider the lenght data as dimension)
I added the filter to exclude the Mary, Queen of the World Cathedral, which has probably a wrong data

In [20]:
queryString = """
SELECT DISTINCT ?cathName ?countryName ?length
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P2043 ?length ;
          wdt:P17 ?country ;
          sc:name ?cathName .
    ?country sc:name ?countryName .
    FILTER (?length < 1500)
}
ORDER BY DESC(?length)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('cathName', "Old St Paul's Cathedral"), ('countryName', 'United Kingdom'), ('length', '586')]


1

### Display together the contruction year and the length 

In [21]:
queryString = """
SELECT DISTINCT ?cathName ?countryName ?costructionYear ?length
WHERE { 
    ?cath wdt:P31 wd:Q2977 ;
          wdt:P571 ?inception ;
          wdt:P17 ?country ;
        wdt:P2043 ?length ;
          sc:name ?cathName .
    BIND(year(xsd:dateTime(?inception)) AS ?costructionYear)
    ?country sc:name ?countryName .
}
ORDER BY DESC(?costructionYear) DESC(?lenght)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('cathName', "St. Michael's Cathedral"), ('countryName', 'Ukraine'), ('costructionYear', '2000'), ('length', '58')]
[('cathName', 'Rio de Janeiro Cathedral'), ('countryName', 'Brazil'), ('costructionYear', '1979'), ('length', '106')]
[('cathName', 'Cathedral of Maringá'), ('countryName', 'Brazil'), ('costructionYear', '1972'), ('length', '50')]
[('cathName', 'Sé Cathedral'), ('countryName', 'Brazil'), ('costructionYear', '1913'), ('length', '111')]
[('cathName', 'Cathedral of Saint Paul, National Shrine of the Apostle Paul'), ('countryName', 'United States of America'), ('costructionYear', '1906'), ('length', '307')]
[('cathName', 'Shrine of the Virgin of the Rosary of Pompei'), ('countryName', 'Italy'), ('costructionYear', '1901'), ('length', '95')]
[('cathName', 'Cathedral of Saint-Jérôme'), ('countryName', 'Canada'), ('costructionYear', '1897'), ('length', '76.9')]
[('cathName', "Saint Mary's Cathedral, Yangon"), ('countryName', 'Myanmar'), ('costructionYear', '1895'), ('le

10

### Result part 4.2
We can notice that there are only one cathedral built in 2000 or later (in our database) and it is also short.
### Note
There are obvious errors of the data, for example [('cathName', 'Mary, Queen of the World Cathedral'), ('countryName', 'Canada'), ('length', '10150')]
As for the height also for the length there may be problem because they are stored as literal (as we can see in the below query) and we have not information about the unit of mesurement.

In [22]:
queryString = """
ASK WHERE { 
    ?cath wdt:P31 wd:Q2977 ;  
        wdt:P2043 ?length .
    
    FILTER isLiteral(?length)
}

"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

### Part 4.3 
If you had to pick a city or a country to visit some cathedrals? Which one would you choose? Based on what criteria?

If I had to pick a city or a country to visit some cathedrals probably I will start choosing the oldest one but not only... In fact we could decide based on the hystorical epoch or the architectural style in which I am interested.